In [4]:
import numpy as np
import random
from scipy.integrate import solve_ivp
import warnings
warnings.filterwarnings('ignore')

class PowerSystemModel:
    def __init__(self, params):
        # System parameters
        self.wn = 376.99111843077515
        self.ws = 1.0
        self.pi = 3.141592653589793

        # Generator parameters
        self.M1, self.M2, self.M3 = 35, 10.0, 4.5
        self.D1, self.D2, self.D3 = 12.0, 8.0, 6.0
        self.T1d1, self.T1d2, self.T1d3 = 8.96, 6.0, 5.89
        self.xd1, self.xd2, self.xd3 = 0.146, 0.8958, 1.3125
        self.xq1, self.xq2, self.xq3 = 0.0969, 0.8645, 1.2578
        self.x1d1, self.x1d2, self.x1d3 = 0.0608, 0.1198, 0.1813

        # UPFC parameters
        self.X_se_upfc = 0.085
        self.b_se_upfc = 11.7647
        self.r_upfc_max = 0.02
        self.r_upfc_min = 0.0001
        self.gamma_max = 5
        self.gamma_min = -5
        self.P_ref_upfc = 0.1
        self.V_ref_upfc = 1.0
        self.T_upfc = 0.5

        # AVR parameters (simplified)
        self.Ka1 = self.Ka2 = self.Ka3 = 20.0
        self.Ta1 = self.Ta2 = self.Ta3 = 0.2
        self.Tr1 = self.Tr2 = self.Tr3 = 0.001
        self.Kf1 = self.Kf2 = self.Kf3 = 0.063
        self.Tf1 = self.Tf2 = self.Tf3 = 0.35
        self.Tef1 = self.Tef2 = self.Tef3 = 0.314

        # Optimizable parameters
        self.Kp_upfc = params[0]
        self.Ki_upfc = params[1]
        self.Kd_upfc = params[2]
        self.D1_extra = params[3]
        self.D2_extra = params[4]
        self.D3_extra = params[5]

        # Initial conditions (from steady state)
        self.V1_0 = 1.04
        self.V2_0 = 1.025
        self.V3_0 = 1.025
        self.delta1_0 = 0.1
        self.delta2_0 = 0.1
        self.delta3_0 = 0.1

        # Setup admittance matrices for proper fault modeling
        self.setup_admittance_matrices()

    def setup_admittance_matrices(self):
        """Setup admittance matrices for different system states"""
        # Normal admittance matrix (9x9) - IEEE 9-bus system
        self.Y_normal = np.zeros((9, 9), dtype=complex)

        # Normal system parameters (simplified IEEE 9-bus)
        # Generator buses (1, 2, 3 -> indices 0, 1, 2)
        # Load buses (4, 5, 6, 7, 8, 9 -> indices 3, 4, 5, 6, 7, 8)

        # Bus 5 (index 4) normal values
        self.Y_normal[4, 4] = 3.2242 - 15.8409j  # G5_5 + jB5_5

        # Bus 7 (index 6) normal values
        self.Y_normal[6, 6] = 2.7722 - 23.3032j  # G7_7 + jB7_7

        # Line 5-7 normal values
        self.Y_normal[4, 6] = -0.5 + 2.0j  # G5_7 + jB5_7
        self.Y_normal[6, 4] = -0.5 + 2.0j  # G7_5 + jB7_5

        # Other buses (simplified)
        for i in range(9):
            if i not in [4, 6]:
                self.Y_normal[i, i] = 2.0 - 10.0j

        # Fault admittance matrix (during fault: t >= 1.0 and t < 1.179)
        self.Y_fault = self.Y_normal.copy()

        # Apply fault at bus 7: G7_7 = 0.0, B7_7 = -1000 (high conductance)
        self.Y_fault[6, 6] = 0.0 - 1000j  # Short circuit at bus 7

        # Post-fault admittance matrix (after t >= 1.179)
        self.Y_post_fault = self.Y_normal.copy()

        # Line 5-7 disconnection
        self.Y_post_fault[4, 6] = 0.0 + 0.0j  # G5_7 = 0, B5_7 = 0
        self.Y_post_fault[6, 4] = 0.0 + 0.0j  # G7_5 = 0, B7_5 = 0

        # Restore normal bus values
        self.Y_post_fault[4, 4] = 3.2242 - 15.8409j  # G5_5, B5_5 restored
        self.Y_post_fault[6, 6] = 2.7722 - 23.3032j  # G7_7, B7_7 restored

    def get_admittance_matrix(self, t):
        """Return appropriate admittance matrix based on time"""
        if 1.0 <= t < 1.179:
            return self.Y_fault  # During fault
        elif t >= 1.179:
            return self.Y_post_fault  # After fault (line disconnected)
        else:
            return self.Y_normal  # Normal operation

    def calculate_network_currents_and_powers(self, V, delta, t):
        """Calculate network currents and electrical powers using admittance matrix"""
        Y = self.get_admittance_matrix(t)

        # Complex voltage vector (9 buses)
        V_complex = np.zeros(9, dtype=complex)

        # Generator buses (simplified voltage calculation)
        V_complex[0] = (self.V1_0 + 0.02 * np.sin(delta[0])) * np.exp(1j * delta[0])  # Bus 1
        V_complex[1] = (self.V2_0 + 0.02 * np.sin(delta[1])) * np.exp(1j * delta[1])  # Bus 2
        V_complex[2] = (self.V3_0 + 0.02 * np.sin(delta[2])) * np.exp(1j * delta[2])  # Bus 3

        # Load buses (simplified)
        for i in range(3, 9):
            V_complex[i] = 1.0 + 0.01 * np.sin(0.1 * t)

        try:
            # Calculate currents: I = Y * V
            I_complex = np.dot(Y, V_complex)

            # Calculate injected powers: S = V * I*
            S_complex = V_complex * np.conj(I_complex)
            P_elec = np.real(S_complex)
            Q_elec = np.imag(S_complex)

            # Return electrical powers for generators (first 3 buses)
            return P_elec[0], P_elec[1], P_elec[2]

        except:
            # Fallback values if calculation fails
            return 1.0, 1.0, 1.0

    def system_dynamics(self, t, y):
        # Extract state variables
        delta1, w1, e1q1, delta2, w2, e1q2, delta3, w3, e1q3 = y[:9]
        r_upfc, gamma_upfc, Vsh_upfc, theta_sh_upfc = y[9:13]
        upfc_error_p, upfc_error_v = y[13:15]

        # Calculate electrical powers using network model with proper fault implementation
        Pe1, Pe2, Pe3 = self.calculate_network_currents_and_powers(
            [self.V1_0, self.V2_0, self.V3_0], [delta1, delta2, delta3], t)

        # Mechanical powers (constant)
        Pm1 = Pm2 = Pm3 = 1.0

        # Generator equations with proper network-based electrical powers
        ddelta1_dt = self.wn * (w1 - self.ws)
        dw1_dt = (1/self.M1) * (Pm1 - Pe1 - (self.D1 + self.D1_extra) * (w1 - self.ws))
        de1q1_dt = (1/self.T1d1) * (-e1q1 - (self.xd1 - self.x1d1) * 1.0 + 1.0)

        ddelta2_dt = self.wn * (w2 - self.ws)
        dw2_dt = (1/self.M2) * (Pm2 - Pe2 - (self.D2 + self.D2_extra) * (w2 - self.ws))
        de1q2_dt = (1/self.T1d2) * (-e1q2 - (self.xd2 - self.x1d2) * 1.0 + 1.0)

        ddelta3_dt = self.wn * (w3 - self.ws)
        dw3_dt = (1/self.M3) * (Pm3 - Pe3 - (self.D3 + self.D3_extra) * (w3 - self.ws))
        de1q3_dt = (1/self.T1d3) * (-e1q3 - (self.xd3 - self.x1d3) * 1.0 + 1.0)

        # UPFC control equations (simplified but with proper fault consideration)
        V4 = 1.0 + 0.005 * (w1 - 1.0)
        V9 = 1.0 + 0.005 * (w3 - 1.0)

        # Power flow calculation considering network changes due to fault
        if 1.0 <= t < 1.179:
            # During fault - reduced power transfer capability
            P_line = 0.3 * V4 * V9 * self.b_se_upfc * np.sin(0.1 * (delta1 - delta3))
        elif t >= 1.179:
            # After fault - line 5-7 disconnected, different power flow pattern
            P_line = 0.7 * V4 * V9 * self.b_se_upfc * np.sin(0.1 * (delta1 - delta3))
        else:
            # Normal operation
            P_line = V4 * V9 * self.b_se_upfc * np.sin(0.1 * (delta1 - delta3))

        dupfc_error_p_dt = (self.P_ref_upfc - P_line) / self.T_upfc
        dupfc_error_v_dt = (self.V_ref_upfc - V9) / self.T_upfc

        # PID control for UPFC
        dr_upfc_dt = (1/self.T_upfc) * (
            self.Kp_upfc * upfc_error_p +
            self.Ki_upfc * (self.P_ref_upfc - P_line) +
            self.Kd_upfc * dupfc_error_p_dt
        )

        dgamma_upfc_dt = (1/self.T_upfc) * (
            self.Kp_upfc * upfc_error_v +
            self.Ki_upfc * (self.V_ref_upfc - V9) +
            self.Kd_upfc * dupfc_error_v_dt
        )

        # Shunt control (simplified)
        dVsh_upfc_dt = (1/self.T_upfc) * (0.1 * (self.V_ref_upfc - V4) - 1.0 * (Vsh_upfc - 1.0))
        dtheta_sh_upfc_dt = (1/self.T_upfc) * (0.02 * (delta1 - delta3) - 0.5 * theta_sh_upfc)

        # Apply limits
        dr_upfc_dt = self._apply_rate_limit(r_upfc, dr_upfc_dt, self.r_upfc_min, self.r_upfc_max)
        dgamma_upfc_dt = self._apply_rate_limit(gamma_upfc, dgamma_upfc_dt, self.gamma_min, self.gamma_max)

        return [ddelta1_dt, dw1_dt, de1q1_dt, ddelta2_dt, dw2_dt, de1q2_dt,
                ddelta3_dt, dw3_dt, de1q3_dt, dr_upfc_dt, dgamma_upfc_dt,
                dVsh_upfc_dt, dtheta_sh_upfc_dt, dupfc_error_p_dt, dupfc_error_v_dt]

    def _apply_rate_limit(self, current_val, rate, min_val, max_val):
        if current_val >= max_val and rate > 0:
            return 0
        elif current_val <= min_val and rate < 0:
            return 0
        return rate

    def simulate(self, t_span=(0, 20), n_points=1000):
        # Initial conditions
        y0 = [self.delta1_0, 1.0, 1.0,  # Gen 1
              self.delta2_0, 1.0, 1.0,  # Gen 2
              self.delta3_0, 1.0, 1.0,  # Gen 3
              0.001, 0.0, 1.0, 0.0,     # UPFC
              0.0, 0.0]                 # Error signals

        t_eval = np.linspace(t_span[0], t_span[1], n_points)

        try:
            sol = solve_ivp(self.system_dynamics, t_span, y0, t_eval=t_eval,
                          method='RK45', rtol=1e-6, atol=1e-8, max_step=0.01)
            return sol.t, sol.y
        except Exception as e:
            print(f"Simulation failed: {e}")
            return None, None

def fitness_function(params):
    """Calculate fitness based on system stability metrics with proper fault modeling"""
    try:
        model = PowerSystemModel(params)
        t, y = model.simulate()

        if t is None or y is None:
            return 1e6  # High penalty for failed simulation

        # Extract generator speeds (w1, w2, w3)
        w1, w2, w3 = y[1], y[4], y[7]

        # Calculate stability metrics

        # 1. Overall speed deviation from nominal
        speed_deviation = np.mean([np.std(w1), np.std(w2), np.std(w3)])

        # 2. Maximum speed excursion during and after fault
        max_speed_deviation = max(np.max(np.abs(w1 - 1.0)),
                                np.max(np.abs(w2 - 1.0)),
                                np.max(np.abs(w3 - 1.0)))

        # 3. Post-fault stability (last 25% of simulation)
        final_period = int(0.25 * len(t))
        final_std = np.mean([np.std(w1[-final_period:]),
                           np.std(w2[-final_period:]),
                           np.std(w3[-final_period:])])

        # 4. Fault response quality (during fault period)
        fault_start_idx = np.where(t >= 1.0)[0][0] if len(np.where(t >= 1.0)[0]) > 0 else 0
        fault_end_idx = np.where(t >= 1.179)[0][0] if len(np.where(t >= 1.179)[0]) > 0 else len(t)

        if fault_end_idx > fault_start_idx:
            fault_response = np.mean([
                np.max(np.abs(w1[fault_start_idx:fault_end_idx] - 1.0)),
                np.max(np.abs(w2[fault_start_idx:fault_end_idx] - 1.0)),
                np.max(np.abs(w3[fault_start_idx:fault_end_idx] - 1.0))
            ])
        else:
            fault_response = 0.1

        # 5. UPFC performance
        r_upfc = y[9]
        upfc_activity = np.std(r_upfc)

        # 6. Recovery time after fault clearing
        if len(t[t >= 1.179]) > 0:
            post_fault_recovery = np.mean([
                np.std(w1[t >= 1.179][:min(500, len(w1[t >= 1.179]))]),
                np.std(w2[t >= 1.179][:min(500, len(w2[t >= 1.179]))]),
                np.std(w3[t >= 1.179][:min(500, len(w3[t >= 1.179]))])
            ])
        else:
            post_fault_recovery = 0.1

        # Combined fitness (lower is better)
        fitness = (speed_deviation * 80 +
                  max_speed_deviation * 60 +
                  final_std * 250 +
                  fault_response * 40 +
                  upfc_activity * 8 +
                  post_fault_recovery * 120)

        # Penalty for unrealistic parameter values
        param_penalty = 0
        if params[0] > 3.0 or params[1] > 1.0:  # Kp, Ki too high
            param_penalty += 150
        if max(params[3:6]) > 1500:  # Damping coefficients too high
            param_penalty += 150
        if min(params) < 0:  # Negative parameters
            param_penalty += 200

        # Stability check - system must remain stable
        if max_speed_deviation > 0.5:  # System becomes unstable
            param_penalty += 500

        return fitness + param_penalty

    except Exception as e:
        return 1e6  # High penalty for any error

class PSO:
    def __init__(self, n_particles=30, n_dimensions=6, bounds=None,
                 max_iter=100, w=0.7, c1=1.5, c2=1.5):
        self.n_particles = n_particles
        self.n_dimensions = n_dimensions
        self.bounds = bounds if bounds else [(-1, 1)] * n_dimensions
        self.max_iter = max_iter
        self.w = w  # Inertia weight
        self.c1 = c1  # Cognitive parameter
        self.c2 = c2  # Social parameter

        # Initialize particles
        self.positions = np.random.uniform(
            [b[0] for b in self.bounds],
            [b[1] for b in self.bounds],
            (n_particles, n_dimensions)
        )

        self.velocities = np.random.uniform(-1, 1, (n_particles, n_dimensions))
        self.personal_best_positions = self.positions.copy()
        self.personal_best_scores = np.full(n_particles, float('inf'))
        self.global_best_position = None
        self.global_best_score = float('inf')

    def optimize(self, fitness_func):
        for iteration in range(self.max_iter):
            # Evaluate fitness for all particles
            for i in range(self.n_particles):
                score = fitness_func(self.positions[i])

                # Update personal best
                if score < self.personal_best_scores[i]:
                    self.personal_best_scores[i] = score
                    self.personal_best_positions[i] = self.positions[i].copy()

                # Update global best
                if score < self.global_best_score:
                    self.global_best_score = score
                    self.global_best_position = self.positions[i].copy()

            # Update velocities and positions
            r1, r2 = np.random.random((2, self.n_particles, self.n_dimensions))

            self.velocities = (self.w * self.velocities +
                             self.c1 * r1 * (self.personal_best_positions - self.positions) +
                             self.c2 * r2 * (self.global_best_position - self.positions))

            self.positions += self.velocities

            # Apply bounds
            for i in range(self.n_dimensions):
                self.positions[:, i] = np.clip(self.positions[:, i],
                                             self.bounds[i][0], self.bounds[i][1])

            # Print progress
            print(f"Iteration {iteration+1}/{self.max_iter}: Best Fitness = {self.global_best_score:.6f}")

            # Early stopping if very good solution found
            #if self.global_best_score < 1.0:
            #    print(f"Early stopping at iteration {iteration+1} - excellent solution found!")
             #   break

        return self.global_best_position, self.global_best_score

# Define parameter bounds
parameter_bounds = [
    (0.1, 2.5),    # Kp_upfc
    (0.01, 0.8),   # Ki_upfc
    (0.0001, 0.02), # Kd_upfc
    (50, 1200),    # D1_extra
    (50, 800),     # D2_extra
    (50, 600)      # D3_extra
]

if __name__ == "__main__":
    print("=" * 60)
    print("PSO OPTIMIZATION FOR IEEE 9-BUS UPFC SYSTEM")
    print("WITH PROPER FAULT IMPLEMENTATION")
    print("=" * 60)

    print("Fault specification:")
    print("- Fault time: t = 1.0s to t = 1.179s")
    print("- Fault type: Short circuit at bus 7 (B7_7 = -1000)")
    print("- Post-fault: Line 5-7 disconnection")
    print()

    print("Current baseline parameters:")
    print(f"Kp_upfc: 0.296764, Ki_upfc: 0.100000, Kd_upfc: 0.000100")
    print(f"D1_extra: 500.00, D2_extra: 200.00, D3_extra: 150.00")
    print("\nStarting optimization...\n")

    # Test baseline fitness
    baseline_params = [0.296764, 0.100000, 0.000100, 500.00, 200.00, 150.00]
    baseline_fitness = fitness_function(baseline_params)
    print(f"Baseline fitness: {baseline_fitness:.6f}")
    print()

    # Initialize and run PSO
    pso = PSO(n_particles=35, n_dimensions=6, bounds=parameter_bounds,
              max_iter=20, w=0.6, c1=1.6, c2=1.4)

    best_params, best_fitness = pso.optimize(fitness_function)

    print("\n" + "=" * 60)
    print("OPTIMIZATION RESULTS")
    print("=" * 60)
    print(f"Best fitness: {best_fitness:.6f}")
    print(f"Improvement: {((baseline_fitness - best_fitness) / baseline_fitness * 100):.2f}%")
    print(f"\nOptimal parameters:")
    print(f"Kp_upfc:  {best_params[0]:.6f}")
    print(f"Ki_upfc:  {best_params[1]:.6f}")
    print(f"Kd_upfc:  {best_params[2]:.6f}")
    print(f"D1_extra: {best_params[3]:.2f}")
    print(f"D2_extra: {best_params[4]:.2f}")
    print(f"D3_extra: {best_params[5]:.2f}")

    # Final verification
    print(f"\nVerifying optimized parameters...")
    final_fitness = fitness_function(best_params)
    print(f"Final verification fitness: {final_fitness:.6f}")

    if final_fitness < baseline_fitness:
        print("✓ Optimization successful - improved system performance!")
    else:
        print("⚠ Optimization did not improve baseline performance")

    print("\nOptimization completed.")

PSO OPTIMIZATION FOR IEEE 9-BUS UPFC SYSTEM
WITH PROPER FAULT IMPLEMENTATION
Fault specification:
- Fault time: t = 1.0s to t = 1.179s
- Fault type: Short circuit at bus 7 (B7_7 = -1000)
- Post-fault: Line 5-7 disconnection

Current baseline parameters:
Kp_upfc: 0.296764, Ki_upfc: 0.100000, Kd_upfc: 0.000100
D1_extra: 500.00, D2_extra: 200.00, D3_extra: 150.00

Starting optimization...

Baseline fitness: 0.760184

Iteration 1/20: Best Fitness = 0.246463
Iteration 2/20: Best Fitness = 0.233878
Iteration 3/20: Best Fitness = 0.212817
Iteration 4/20: Best Fitness = 0.202022
Iteration 5/20: Best Fitness = 0.202022
Iteration 6/20: Best Fitness = 0.202022
Iteration 7/20: Best Fitness = 0.202022
Iteration 8/20: Best Fitness = 0.202022
Iteration 9/20: Best Fitness = 0.202022
Iteration 10/20: Best Fitness = 0.202022
Iteration 11/20: Best Fitness = 0.202022
Iteration 12/20: Best Fitness = 0.202022
Iteration 13/20: Best Fitness = 0.202022
Iteration 14/20: Best Fitness = 0.202022
Iteration 15/20: 